# Clase Práctica 02

# Neural Network - Redes Neuronales Artificiales

# Evaluación del rendimiento de los modelos


# Introducción

Se deben tomar muchas decisiones al diseñar y configurar sus modelos de Deep Learning. La mayoría de estas decisiones deben resolverse empíricamente a través de prueba y error, y evaluándolas en **datos reales**. Como tal, es de vital importancia tener una forma sólida de evaluar el rendimiento de su red neuronal y los modelos de aprendizaje profundo.

Hay una gran cantidad de decisiones que debe tomar al diseñar y configurar sus modelos de aprendizaje profundo. Muchas de estas decisiones pueden resolverse copiando la estructura de las redes de otras personas y utilizando heurísticas. En última instancia, la mejor técnica es **diseñar experimentos pequeños y evaluar empíricamente las opciones utilizando datos reales**. Esto incluye decisiones de alto nivel como el número, tamaño y tipo de capas en su red. También incluye las decisiones de nivel inferior, como la elección de la función de pérdida, las funciones de activación, el optimizador y el número de épocas.



# Separación de los datos 

La gran cantidad de datos y la complejidad de los modelos requieren tiempos de entrenamiento muy largos. Como tal, normalmente se usa una separación simple de datos, generalmente las bases de datos se dividen en conjuntos de datos de entrenamiento y prueba (test), o conjuntos de datos de entrenamiento y validación. Keras proporciona dos formas convenientes de evaluar sus algoritmos de aprendizaje profundo:

1. Usar un conjunto de datos de separación automática.
2. Usar un conjunto de datos de separación manual.



# Separación automática

Keras puede separar una parte de sus datos en un conjunto de datos de entrenamiento y validación, y evaluar el rendimiento de su modelo en ese conjunto de datos de validación en cada época. La separación automática se realiza configurando el argumento "validation_split" en la función fit(), indicando un porcentaje de separaciós de los datos. Un valor razonable podría ser 0.2 o 0.33 para indicar el 20% o el 33% de sus datos de entrenamiento retenidos para su validación (ej. 20% indica 80% de datos de entrenamiento y 20% para validación). El siguiente ejemplo demuestra el uso del uso de un conjunto de datos de validación automática en el inicio de datos de diabetes vista en la primera práctica.


In [ ]:
#librerías
from keras.models import Sequential
from keras.layers import Dense
import numpy
# seed
seed = 7
numpy.random.seed(seed)
# base de datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# entradas y salidas
X = dataset[:,0:8]
Y = dataset[:,8]
# modelo
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# compilar
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# ajuste
model.fit(X, Y, validation_split=0.33, nb_epoch=150, batch_size=10) #  33% para validación  


# Separación manual

Keras también permite especificar manualmente el conjunto de datos que se usará para la validación durante el entrenamiento. En este ejemplo, usamos la función train_test_split() de la biblioteca de Scikit-Learn de Python para separar nuestros datos en un conjunto de datos de entrenamiento y test. Utilizamos el 67% para el entrenamiento y el 33% restante de los datos para la validación. El conjunto de datos de validación se puede especificar en la función fit() en Keras mediante el argumento validation_data. 


In [ ]:
# librerías
from keras.models import Sequential
from keras.layers import Dense
from sklearn.cross_validation import train_test_split
import numpy
# seed
seed = 7
numpy.random.seed(seed)
# datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# entradas y salidas
X = dataset[:,0:8]
Y = dataset[:,8]
# separar datos manualmente 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=seed)
# modelo
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# compilar
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# ajuste
model.fit(X_train, y_train, validation_data=(X_test,y_test), nb_epoch=150, batch_size=10) # se valida con x_test, y_test

In [ ]:
# evaluar solo con datos de test (datos no vistos por la red)
scores = model.evaluate(X_test, y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


# Validación cruzada manual de K-Fold (Cross Validation)

El estándar de oro para la evaluación de modelos de Deep Learning es la validación cruzada k-fold. Proporciona una estimación robusta del rendimiento de un modelo en datos invisibles (datos nuevos o datos no vistos por la red). Para ello, el método k-fold divide el conjunto de datos de entrenamiento en k subconjuntos y toma turnos para los modelos de entrenamiento en todos los subconjuntos, excepto uno que se mantiene. Luego, se evalúa el rendimiento del modelo entrenado con el conjunto de datos de validación retenido. El proceso se repite hasta que todos los subconjuntos tengan la oportunidad de ser el conjunto de validación retenido. La medida de rendimiento se promedia en todos los modelos que se crean, obteniendo un valor promedio final.

La validación cruzada posee un alto costo computacional. Por ejemplo, la validación cruzada de k-fold se usa a menudo con un k de 5 o 10 veces. Como tal, se deben construir y evaluar 5 ó 10 modelos, lo que aumenta considerablemente el tiempo de evaluación de un modelo. Sin embargo, cuando el problema es lo suficientemente pequeño o si se tienen suficientes recursos de cómputo, la validación cruzada k-fold puede darle una estimación **menos sesgada** del rendimiento de su modelo.

En el siguiente ejemplo, utilizamos la funcón StratifiedKFold de la biblioteca Scikit-Learn para dividir el conjunto de datos de entrenamiento en 10 carpetas. Las carpetas están **estratificadas**, lo que significa que el algoritmo intenta equilibrar el número de instancias de cada clase en cada pliegue. El ejemplo crea y evalúa 10 modelos utilizando las 10 divisiones de los datos y recopila todas las puntuaciones (precisión). Se desactiva el visualizador de datos al usar verbose = 0 en las funciones fit () y evaluar () en el modelo. El rendimiento se imprime para cada modelo y se almacena. El promedio y la desviación estándar del rendimiento del modelo se imprimen al final de la ejecución para proporcionar una estimación sólida de la precisión del modelo.

In [ ]:
#librerías
from keras.models import Sequential
from keras.layers import Dense
from sklearn.cross_validation import StratifiedKFold
import numpy
# seed
seed = 2
numpy.random.seed(seed)
# datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# entradas y salidas
X = dataset[:,0:8]
Y = dataset[:,8]
# definir 10-fold
kfold = StratifiedKFold(y=Y, n_folds=2, shuffle=True, random_state=seed)
# para guardar los puntajes
cvscores = []
# ciclo para los 10-fold
for i, (train, test) in enumerate(kfold):
    # modelo
    model = Sequential()
    model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
    model.add(Dense(8, init='uniform', activation='relu'))
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    # compilar
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # ajustar
    model.fit(X[train], Y[train], nb_epoch=150, batch_size=10, verbose=0)
    # evaluar
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))


# Guardar modelos

Dado que los modelos de Deep Learning pueden tomar horas, días e incluso semanas para entrenarse, es importante saber cómo guardarlos y cargarlos desde el disco. A continuación, se mostrará cómo puede guardar sus modelos Keras para cargarlos nuevamente para hacer predicciones.

Keras separa la forma de guardar la arquitectura de su modelo y guardar los pesos de su modelo. Los pesos de los modelos se guardan en formato HDF5. Este es un formato que es ideal para almacenar matrices de números multidimensionales. La estructura del modelo se puede describir y guardar (y cargar) utilizando dos formatos diferentes: JSON y YAML.



# Formato HDF5

El formato de datos jerárquico o HDF5, es un formato de almacenamiento de datos flexible y es conveniente para almacenar grandes arreglos de valores reales, como los que tenemos en los pesos de las redes neuronales. Es posible que deba instalar el soporte de Python para el formato de archivo HDF5. Puede hacer esto utilizando su sistema de administración de paquetes Python preferido, como pip:

pip install h5py



# Guardar modelo de red neuronal en formato JSON

JSON es un formato de archivo simple para describir datos jerárquicamente. Keras proporciona la capacidad de describir cualquier modelo utilizando el formato JSON con una función to_json(). Puede guardarse en un archivo y luego cargarse a través del modelo desde la función model_from_json(), que creará un nuevo modelo.

Los pesos se guardan directamente utilizando la función guardar pesos "save_weights()" y pueden ser cargados posteriormente utilizando la función de pesos de carga "load_weights()". 

El siguiente ejemplo entrena y evalúa un modelo simple con la base de datos Pima Indians. La estructura del modelo se convierte a continuación en formato JSON y se escribe el modelo en model.json en el directorio local. Los pesos de red se escriben en model.h5 en el directorio local.

Los datos del modelo y el peso se cargan desde los archivos guardados y se crea un nuevo modelo. Es importante compilar el modelo cargado antes de utilizarlo. Esto es así para que las predicciones realizadas con el modelo puedan usar el cálculo del backend de Keras. El modelo se evalúa de la misma manera imprimiendo el mismo puntaje de evaluación.

In [ ]:
#librerías
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os
# seed
seed = 7
numpy.random.seed(seed)
# datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# entradas y salidas
X = dataset[:,0:8]
Y = dataset[:,8]
# modelo
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# compilar
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# ajustar
model.fit(X, Y, nb_epoch=150, batch_size=10, verbose=0)
# evaluar
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# convertir model a JSON
model_json = model.to_json()
with open("model_guardado.json", "w") as json_file:
    json_file.write(model_json)
    
# convertir weights a HDF5
model.save_weights("model_guardado.h5")
print("Guardar modelo al disco")


# cargando el modelo desde el disco

json_file = open('model_guardado.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# cargar los pesos del nuevo modelo
loaded_model.load_weights("model_guardado.h5")
print("Cargar modelo del disco")

# evaluar el modelo cargado desde el disco con los datos 
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))



# Guardar modelo de red neuronal en formato YAML

Este ejemplo es muy similar al ejemplo JSON anterior, excepto que se usa el formato YAML para la especificación del modelo. El modelo se describe utilizando YAML, se guarda en el model.yaml y luego se carga en un nuevo modelo a través del modelo de la función yaml(). Los pesos se manejan de la misma manera que en el formato HDF5 que en el modelo.h5.

In [ ]:
# librerías
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_yaml
import numpy
import os
# seed
seed = 7
numpy.random.seed(seed)
# datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# entradas y salidas
X = dataset[:,0:8]
Y = dataset[:,8]
# modelo
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# compilar
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# ajuste
model.fit(X, Y, nb_epoch=150, batch_size=10, verbose=0)
# evaluar
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


# convertir modelo a YAML
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
    
# convertir pesos a HDF5
model.save_weights("model.h5")
print("Saved model to disk")



# cargar el modelo en formato YAML desde el disco

yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# cargar pesos del disco
loaded_model.load_weights("model.h5")

print("Cargar modelo del disco")
# evaluar modelo 
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

# Checkpointing para modelos de redes neuronales

El punto de control es una técnica de tolerancia a fallas para procesos de larga ejecución. Es un enfoque en el que se toma una (foto) instantánea del estado del sistema en caso de falla. Si hay un problema, no todo está perdido. El punto de control se puede usar directamente como punto de partida, retomando donde quedó. Cuando se entrenan modelos de Deep Learning, el punto de control permite monitorear y capturar los pesos del modelo. 

Keras proporciona el punto de control mediante una API de devolución de llamada (Checkpoint). La clase de devolución de llamada "ModelCheckpoint" le permite definir dónde marcar los pesos del modelo, cómo se debe nombrar el archivo y en qué circunstancias realizar un punto de control del modelo. La API le permite especificar qué métrica controlar, como la pérdida o la precisión en el conjunto de datos de entrenamiento o validación. Puede especificar si desea buscar una mejora en maximizar o minimizar la puntuación (score). Finalmente, el nombre que utiliza para almacenar los pesos puede incluir variables como el número de épocas o la métrica. La instancia de ModelCheckpoint se puede pasar al proceso de entrenamiento al llamar a la función de ajuste fit() en el modelo.


# Checkpointing para mejorar modelos neuronales

Un buen uso del punto de control **Checkpoint** es generar los pesos del modelo cada vez que se observa una mejora durante el entrenamiento. El siguiente ejemplo crea una pequeña red neuronal para el problema de clasificación binaria de la base Pima Indians. El ejemplo utiliza el 33% de los datos para la validación.

El punto de control se configura para guardar los pesos de la red solo cuando hay **una mejora en la precisión** de la clasificación en el conjunto de datos de validación (monitor = 'val acc' y mode = 'max'). Los pesos se almacenan en un archivo que incluye la puntuación en el nombre de archivo: weights-improvement-val acc=.2f.hdf5.

In [ ]:
# librerías
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint   # se agrega el callbacks de Keras 
import matplotlib.pyplot as plt
import numpy
# seed
seed = 7
numpy.random.seed(seed)
# datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# entradas y salidas
X = dataset[:,0:8]
Y = dataset[:,8]
# modelo
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# compilar
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"    # nombre asignado solo cuando haya mejora en los pesos
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max') # checkpoint
callbacks_list = [checkpoint] # lista de checkpoint solicitados, en este caso solo guardar pesos cuando haya mejora
# ajuste que incluye el callback 
model.fit(X, Y, validation_split=0.33, nb_epoch=150, batch_size=10, callbacks=callbacks_list, verbose=0)

# note que cada vez que haya una mejora en precisión de la validación se guardarán los pesos...


# Checkpointing para el mejor modelo de red neuronal

Una estrategia de Checkpoint más simple es guardar los pesos del modelo en el mismo archivo, solo si la precisión de la validación mejora. Esto se puede hacer fácilmente usando el mismo código de arriba y cambiando el nombre del producto a ser uno fijo (no incluye información de puntuación o de época). En este caso, los pesos del modelo se escriben en el archivo weight.best.hdf5 solo si la precisión de la clasificación del modelo en los datos validación mejora con respecto a la mejor vista hasta el momento.

In [ ]:
# librerías
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy
# seed
seed = 7
numpy.random.seed(seed)
# datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# entradas y salidas
X = dataset[:,0:8]
Y = dataset[:,8]
# modelo
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# compilar
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# checkpoint
filepath="weights.best.hdf5"    # nombre fijo, se reemplazan los pesos constantemente
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# ajuste
model.fit(X, Y, validation_split=0.33, nb_epoch=150, batch_size=10, callbacks=callbacks_list, verbose=0)

# Cargar un modelo de red neuronal guardada desde el Checkpoint

Ahora que ha visto cómo controlar sus modelos de Deep Learning durante el entrenamiento, debe revisar cómo cargar y usar un modelo desde un Checkpoint. El Checkpoint solo incluye los pesos del modelo. **Se supone que usted conoce la estructura de la red**. Esto también puede ser convertirdo a archivos en formato JSON o YAML. 

En el siguiente ejemplo, la estructura del modelo es conocida y se cargan los mejores pesos del experimento anterior, almacenados en el directorio de trabajo en el archivo "weight.best.hdf5". El modelo se utiliza para hacer predicciones en todo el conjunto de datos.

In [ ]:
# librerías
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy
# seed
seed = 7
numpy.random.seed(seed)
# modelo
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# cargar pesos
model.load_weights("weights.best.hdf5")
# compilar es necesario para realizar las predicciones
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Created model and loaded weights from file")
# datos 
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# entradas y salidas
X = dataset[:,0:8]
Y = dataset[:,8]
# estimar precisión con los pesos cargados
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# Acceso a la historia del modelo 

Keras proporciona la capacidad de registrar devoluciones de llamada cuando se entrena un modelo de Deep Learning. Una de las devoluciones de llamada predeterminadas que se registra cuando se entrenan todos los modelos de Deep Learning es la devolución de llamada del historial (History). La historia registra métricas de entrenamiento para cada época. Esto incluye la pérdida y la precisión (para problemas de clasificación), así como la pérdida y la precisión del conjunto de datos de validación, si se establece uno. La historia se devuelve desde las llamadas a la función fit() que se usa para entrenar el modelo.

Las métricas se almacenan en un diccionario. Por ejemplo, puede enumerar las métricas recopiladas en la historia utilizando el siguiente fragmento de código después de que se haya entrenado un modelo:

print(history.history.keys())

que produce el siguiente listado:

['acc', 'loss', 'val_acc', 'val_loss']


# Visualizar la historia del modelo

Podemos crear gráficos a partir de los datos históricos recogidos. En el siguiente ejemplo, creamos una pequeña red para modelar el problema de clasificación binaria de Pima Indians. El ejemplo recopila el historial, entregando el modelo de entrenamiento y crea dos figuras:

1. Un gráfico de precisión en los conjuntos de datos de entrenamiento y validación durante las épocas de entrenamiento.
2. Un gráfico de la pérdida en los conjuntos de datos de entrenamiento y validación durante las épocas de entrenamiento.



In [ ]:
# librerías
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy
# seed
seed = 7
numpy.random.seed(seed)
# datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# entradas y salidas
X = dataset[:,0:8]
Y = dataset[:,8]
# modelo
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# compilar
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# ajuste 
history = model.fit(X, Y, validation_split=0.33, nb_epoch=150, batch_size=10, verbose=0) # guardamos la historia
# mostrar los datos que tiene historia
print(history.history.keys())

# gráfico para la precisión obtenido de los datos de la historia 

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'ro', label='Acc Entrenamiento')
plt.plot(epochs, val_acc, 'b', label='Acc Validación')
plt.title('Accuracy: Entrenamiento and Validación')
plt.legend()
plt.savefig("Fig1.png")

# gráfico para la pérdida obtenido de los datos de la historia 
plt.figure()
plt.plot(epochs, loss, 'ro', label='Loss Entrenamiento')
plt.plot(epochs, val_loss, 'b', label='Loss Validación')
plt.title('Loss: Entrenamiento and Validación')
plt.legend()
plt.savefig("Fig2.png")
plt.show()


In [ ]:
numpy.mean(acc)

# Parámetros de los modelos de Deep Learning: búsqueda de cuadrícula (grid)

En este ejemplo, utilizamos una búsqueda de cuadrícula (grid) para evaluar diferentes configuraciones para nuestro modelo de red neuronal e informar sobre la combinación que proporciona el mejor rendimiento estimado. 

Vamos a crear una función, llamada crear_model(), que le permite tomar dos argumentos, el optimizador y el inicializador (optimizer, init). Esto nos permitirá evaluar el efecto de utilizar diferentes algoritmos de optimización y esquemas de inicialización de los pesos para nuestra red. Después de crear nuestro modelo, vamos a definir una serie de valores que pueden tomar los parámetros que deseamos buscar, específicamente:

1. Se usarán diferentes Optimizadores para la búsqueda de diferentes valores de los pesos.

2. Se usarán diferentes Inicializadores para preparar los pesos de red utilizando diferentes esquemas.

3. Se usarán diferentes Número de épocas para el entrenamiento del modelo para diferentes exposiciones al entrenamiento de conjunto de datos

4. Se usarán diferentes Lotes para variar el número de muestras antes de las actualizaciones de peso.

Las opciones se especifican en un diccionario y se pasan a la configuración de la clase **GridSearchCV** de Scikit-Learn. 

La clase evaluará una versión de nuestro modelo de red neuronal para cada combinación de parámetros (2 x 3 x 3 x 3) definidos como las combinaciones de diferentes optimizadores, inicializaciones, épocas y lotes. Luego, cada combinación se evalúa utilizando el valor predeterminado de la validación cruzada (3 carpetas).

Eso corresponde a un montón de modelos y una gran cantidad de cálculo mediante el computador. Este no es un esquema que quiera usar a la ligera debido al tiempo que tomará realizar los cálculos. Puede ser útil para usted diseñar pequeños experimentos con un subconjunto más pequeño de sus datos, para tener tiempos razonables. Este ejemplo es un experimento razonable, ya que en este caso se usa una pequeña red y un pequeño conjunto de datos (menos de 1,000 instancias y 9 atributos). Finalmente, se muestran el rendimiento y la combinación de configuraciones para el mejor modelo, seguido del rendimiento de todas las combinaciones de parámetros.


In [ ]:
# librerías
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier   # se agrega KerasClassifiear como contenedor de la red
from sklearn.model_selection import GridSearchCV     # se agrega GridSearchCV
import numpy

# funcion para crear el modelo de red, se utiliza con KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, init=init, activation='relu'))
    model.add(Dense(8, init=init, activation='relu'))
    model.add(Dense(1, init=init, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# seed
seed = 4
numpy.random.seed(seed)

# datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# entradas y salidas
X = dataset[:,0:8]
Y = dataset[:,8]

# modelo, note que se cargará un modelo desde un contenedor que tendrá diferentes parámetros
model = KerasClassifier(build_fn=create_model)

#parámetros de la GRID que usaremos para buscar el mejor rendimiento 
# epochs, batch size y optimizer
optimizers = ['rmsprop', 'adam']                          # dos optimizadores diferentes para probar el modelo
init = ['glorot_uniform', 'normal', 'uniform']          # tres inicializadores diferentes
epochs = numpy.array([2, 5, 10])                        # tres valores de epocas diferentes 
batches = numpy.array([5, 10, 20])                      # tres valores de batches diferentes
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)   # crea un diccionario
grid = GridSearchCV(estimator=model, param_grid=param_grid)  # genera la GRID para diferentes parámetros 
grid_result = grid.fit(X, Y)                                # ajusta los modelos con diferentes parametros 

print("Mejor Modelo: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))